# Microbiome data analysis using QIIME 2
### Name: Azad and Batarilo
### Date: 2025-12-14
### GitHub Repo: https://github.com/mitrazad/QIIME2_Workflow
This notebook documents a complete and reproducible microbiome analysis workflow using QIIME 2. Paired-end 16S rRNA gene amplicon sequencing data were processed to generate both amplicon sequence variants (ASVs) and operational taxonomic units (OTUs). The analysis includes quality control, contaminant removal using negative controls, taxonomic assignment, diversity analyses (phylogenetic and non-phylogenetic), and differential abundance testing.

All analysis steps are documented, intermediate outputs are saved, and results are suitable for downstream interpretation and reporting.

## 0. Set-Up

Inputs (already present):
```sql
project/
├─ raw/                      # demultiplexed paired-end FASTQ(.gz) files
│  ├─ <sample>_1.fastq.gz
│  └─ <sample>_2.fastq.gz
├─ ref/                      # reference data
│  └─ silva-138-99-nb-classifier.qza
└─ metadata/
   └─ sample-metadata.tsv    # sample sheet (incl. columns like role, disease-state, type)
```

Outputs (created by this notebook):
```sql
project/
├─ qiime2/                   # QIIME 2 artifacts (.qza) and visualizations (.qzv)
│  ├─ diversity-asv/
│  └─ diversity-otu/
└─ metadata/
   └─ sample-metadata.qzv

In [1]:
from pathlib import Path
import os
import sys
import platform
import qiime2

PROJECT_DIR = Path.cwd()
RAW_DIR = PROJECT_DIR / "raw"
REF_DIR = PROJECT_DIR / "ref"
META_DIR = PROJECT_DIR / "metadata"

os.makedirs(PROJECT_DIR / "qiime2", exist_ok=True)
os.makedirs(PROJECT_DIR / "qiime2/diversity-asv", exist_ok=True)
os.makedirs(PROJECT_DIR / "qiime2/diversity-otu", exist_ok=True)

print("=== Environment ===")
print("QIIME 2:", qiime2.__version__)
print("Python :", sys.version.split()[0])
print("OS     :", platform.platform())

print("\n=== Project paths ===")
print("PROJECT_DIR:", PROJECT_DIR)
print("RAW_DIR    :", RAW_DIR)
print("REF_DIR    :", REF_DIR)
print("META_DIR   :", META_DIR)

# Basic sanity checks
if not RAW_DIR.exists():
    raise FileNotFoundError(f"Missing raw/ directory at: {RAW_DIR}")
if not (REF_DIR / "silva-138-99-nb-classifier.qza").exists():
    raise FileNotFoundError("Missing classifier: ref/silva-138-99-nb-classifier.qza")
if not (META_DIR / "sample-metadata.tsv").exists():
    raise FileNotFoundError("Missing metadata: metadata/sample-metadata.tsv")


=== Environment ===
QIIME 2: 2025.7.0
Python : 3.10.14
OS     : Linux-5.15.0-139-generic-x86_64-with-glibc2.31

=== Project paths ===
PROJECT_DIR: /home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipynb
RAW_DIR    : /home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipynb/raw
REF_DIR    : /home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipynb/ref
META_DIR   : /home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipynb/metadata


## 1. Metadata tabulation

Load and inspect sample metadata to ensure that all required variables (e.g. disease status, sample type, control annotation) are present and correctly formatted.

**Output:** `metadata/sample-metadata.qzv`

In [2]:
from qiime2 import Metadata
from qiime2.plugins import metadata as q2_metadata

# Load data
sample_metadata = Metadata.load(META_DIR / "sample-metadata.tsv")

# Metadata summary
metadata_viz = q2_metadata.visualizers.tabulate(input=sample_metadata)
metadata_viz.visualization.save(META_DIR / "sample-metadata.qzv")

/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


'/home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipynb/metadata/sample-metadata.qzv'

## 2. Create manifest

Create a manifest file linking sample IDs to absolute paths of paired-end FASTQ files (forward and reverse read file paths), which is necessary for import into QIIME 2.
This step also performs basic consistency checks to ensure that:
- Each sample has both forward and reverse reads
- File naming follows the expected convention
- No samples are silently skipped during import

**Output:** `metadata/manifest.tsv`

In [3]:
# Define output path
OUT = META_DIR / "manifest.tsv"
OUT.parent.mkdir(parents=True, exist_ok=True)

# Identify forward read files
fwd_files = sorted(RAW_DIR.glob("*_1.fastq.gz"))
if not fwd_files:
    raise SystemExit(f"ERROR: No *_1.fastq.gz files found in {RAW_DIR}")

rows = []
for fwd in fwd_files:
    # Derive sample ID from filename
    sample_id = fwd.name.replace("_1.fastq.gz", "")
    
    # Define corresponding reverse read
    rev = RAW_DIR / f"{sample_id}_2.fastq.gz"
    if not rev.exists():
        raise SystemExit(f"ERROR: Missing reverse read for sample '{sample_id}': expected {rev}")
    
    # Store absolute file paths
    rows.append((sample_id, str(fwd.resolve()), str(rev.resolve())))

# Write manifest file
with OUT.open("w", encoding="utf-8") as fh:
    fh.write("sample-id\tforward-absolute-filepath\treverse-absolute-filepath\n")
    for sample_id, fwd, rev in rows:
        fh.write(f"{sample_id}\t{fwd}\t{rev}\n")
        
print(f"Manifest written to: {OUT}")
print(f"Samples in manifest: {len(rows)}")
print((META_DIR / "manifest.tsv").read_text())

Manifest written to: /home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipynb/metadata/manifest.tsv
Samples in manifest: 23
sample-id	forward-absolute-filepath	reverse-absolute-filepath
EF1	/home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipynb/raw/EF1_1.fastq.gz	/home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipynb/raw/EF1_2.fastq.gz
EF2	/home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipynb/raw/EF2_1.fastq.gz	/home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipynb/raw/EF2_2.fastq.gz
EF3	/home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipynb/raw/EF3_1.fastq.gz	/home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipynb/raw/EF3_2.fastq.gz
EF4	/home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipynb/raw/EF4_1.fastq.gz	/home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipynb/raw/EF4_2.fastq.gz
EP1	/home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipynb/raw/EP1_1.fastq.gz	/home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipynb/raw/EP1_2.fastq.gz
EP2	/home/fhwn.ac.at/211933/251214_Fallstudie/qiime2_ipy

In [4]:
md_ids = set(sample_metadata.to_dataframe().index.astype(str))
manifest_ids = {r[0] for r in rows}
missing_in_md = sorted(manifest_ids - md_ids)
if missing_in_md:
    raise ValueError(
        "Samples present in FASTQs/manifest but missing in metadata. "
        f"First 10: {missing_in_md[:10]}"
    )

## 3. Import paired-end FASTQs

**Purpose**: Import the paired-end FASTQ files into QIIME 2 format and generate an initial quality-control summary of the sequencing data.

The manifest created in the previous step tells QIIME 2 exactly which forward and reverse FASTQ files belong to each sample. Importing converts these FASTQs into a SampleData[PairedEndSequencesWithQuality] artifact, which can then be used for primer trimming (cutadapt) and denoising (DADA2).

After import, the demultiplexing summary provides:
- Per-sample sequence counts (to detect low-depth or failed samples)
- Per-base quality score plots for forward and reverse reads (to guide truncation settings for DADA2)

Output: `paired-end-demux.qza` + `.qzv`

In [5]:
from qiime2 import Artifact
from qiime2.plugins import demux

# Import paired-end reads using the manifest file
paired_end_demux = Artifact.import_data(
    type="SampleData[PairedEndSequencesWithQuality]",
    view_type="PairedEndFastqManifestPhred33V2",
    view=str(META_DIR / "manifest.tsv")
)

# Save imported artifact
paired_end_demux.save("qiime2/paired-end-demux.qza")

# Demultiplexing / quality summary
demux_summary = demux.visualizers.summarize(data=paired_end_demux)
demux_summary.visualization.save("qiime2/paired-end-demux.qzv")

/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_demux/_summarize/_visualizer.py:182: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  context['result_data'] = pd.concat([context['result_data'], df])
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_demux/_summarize/_visualizer.py:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pand

'qiime2/paired-end-demux.qzv'

## 4. Primer trimming

**Purpose**: Remove locus-specific 16S rRNA gene primer sequences from the paired-end reads before denoising.

Sequencing reads still contain the V3–V4 locus-specific primers (341F/805R). These non-biological sequences must be removed before DADA2 denoising to avoid artificial sequence variants and biased diversity estimates.

Primer sequences (locus-specific):
- Forward primer (341F): `CCTACGGGNGGCWGCAG`
- Reverse primer (805R): `GACTACHVGGGTATCTAATCC`

Key parameters:
- error_rate=0.1: allows mismatches (important for degenerate primers and sequencing error)
- minimum_length=200: avoids keeping overly short reads after trimming
- discard_untrimmed=True: removes reads without detectable primer sequence

**Output:** `qiime2/paired-end-demux-trimmed.qza`

In [6]:
from qiime2.plugins import cutadapt

FWD_PRIMER = "CCTACGGGNGGCWGCAG"
REV_PRIMER = "GACTACHVGGGTATCTAATCC"

# Trim locus-specific primers from paired-end reads
trimmed = cutadapt.methods.trim_paired(
    demultiplexed_sequences=paired_end_demux,
    front_f=[FWD_PRIMER],
    front_r=[REV_PRIMER],
    error_rate=0.1,
    match_read_wildcards=True,
    match_adapter_wildcards=True, 
    minimum_length=200,
    discard_untrimmed=True,
    cores = 0
)

# Save trimmed reads as a QIIME 2 artifact
trimmed.trimmed_sequences.save("qiime2/paired-end-demux-trimmed.qza")

# Summarize trimmed reads (counts + quality after trimming)
trimmed_summary = demux.visualizers.summarize(data=trimmed.trimmed_sequences)
trimmed_summary.visualization.save("qiime2/paired-end-demux-trimmed.qzv")

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt -u 0 --error-rate 0.1 --times 1 --overlap 3 --minimum-length 200 -q 0,0 --quality-base 33 --cores 0 -o /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-tkpxa2c7/EF1_0_L001_R1_001.fastq.gz -p /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-tkpxa2c7/EF1_23_L001_R2_001.fastq.gz --front CCTACGGGNGGCWGCAG -G GACTACHVGGGTATCTAATCC -U 0 --match-read-wildcards --discard-untrimmed /tmp/qiime2/211933/data/d99a659a-b928-4e1d-8fb5-d66a59efc97f/data/EF1_0_L001_R1_001.fastq.gz /tmp/qiime2/211933/data/d99a659a-b928-4e1d-8fb5-d66a59efc97f/data/EF1_23_L001_R2_001.fastq.gz

This is cutadapt 5.1 with Python 3.10.14
Command line parameters: -u 0 --error-rate 0.1 --times 1 --overlap 3 --minimum-length 200 -q 0,0 

/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_demux/_summarize/_visualizer.py:182: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  context['result_data'] = pd.concat([context['result_data'], df])
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_demux/_summarize/_visualizer.py:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pand

'qiime2/paired-end-demux-trimmed.qzv'

## 5. DADA2 denoising (ASVs)

**Purpose:** Infer high-resolution amplicon sequence variants (ASVs) by correcting sequencing errors, merging paired-end reads, and removing chimeras.

DADA2 uses a model-based approach to distinguish true biological sequence variants from sequencing errors. Unlike OTU clustering, this approach resolves sequences at single-nucleotide resolution, improving reproducibility and comparability across studies.

In this workflow, DADA2 is applied after primer trimming, ensuring that non-biological primer sequences do not inflate diversity estimates or generate artificial ASVs.

Truncation lengths were selected based on the post-trimming quality profiles:
- Forward reads truncated at 270 bp
- Reverse reads truncated at 230 bp

The expected V3–V4 amplicon length is ~460 bp. With these truncation settings, the retained reads provide an overlap of approximately 40 bp, which is sufficient for accurate paired-end merging while removing low-quality bases at the read tails. No additional fixed trimming was applied during denoising (trim_left_f = trim_left_r = 0) because primers were already removed using cutadapt.

**Output:** `qiime2/`
- ASV feature table: `table-asv.qza`
- Representative ASV sequences: `rep-seqs-asv.qza`
- DADA2 denoising statistics: `stats-dada2-asv.qza` + `.qzv`

In [8]:
import pandas as pd
from qiime2.plugins import dada2

# DADA2 denoising on primer-trimmed paired-end reads
dada2_results = dada2.methods.denoise_paired(
    demultiplexed_seqs=trimmed.trimmed_sequences,
    trunc_len_f=270,
    trunc_len_r=230,
    trim_left_f=0,
    trim_left_r=0,
    n_threads=0
)
print(dada2_results.table.view(pd.DataFrame).shape)

# Save DADA2 outputs
dada2_results.table.save("qiime2/table-asv.qza")
dada2_results.representative_sequences.save("qiime2/rep-seqs-asv.qza")
dada2_results.denoising_stats.save("qiime2/stats-dada2-asv.qza")

# Visualize DADA2 denoising statistics
stats_viz = q2_metadata.visualizers.tabulate(input=dada2_results.denoising_stats.view(Metadata))
stats_viz.visualization.save("qiime2/stats-dada2-asv.qzv")

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada.R --input_directory /tmp/tmp_vn7lmlu/forward --input_directory_reverse /tmp/tmp_vn7lmlu/reverse --output_path /tmp/tmp_vn7lmlu/output.tsv.biom --output_track /tmp/tmp_vn7lmlu/track.tsv --filtered_directory /tmp/tmp_vn7lmlu/filt_f --filtered_directory_reverse /tmp/tmp_vn7lmlu/filt_r --truncation_length 270 --truncation_length_reverse 230 --trim_left 0 --trim_left_reverse 0 --max_expected_errors 2.0 --max_expected_errors_reverse 2.0 --truncation_quality_score 2 --min_overlap 12 --max_merge_mismatch 0 --trim_overhang False --pooling_method independent --chimera_method consensus --min_parental_fold 1.0 --allow_one_off False --num_threads 0 --learn_min_reads 1000000

R version 4.3.3 (2024-02-29) 


Loading required package: Rcpp


DADA2: 1.30.0 / Rcpp: 1.1.0 / RcppParallel: 5.1.9 
2) Filtering .......................
3) Learning Error Rates
281994480 total bases in 1044424 reads from 22 samples will be used for learning the error rates.
240217520 total bases in 1044424 reads from 22 samples will be used for learning the error rates.
3) Denoise samples .......................
.......................
5) Remove chimeras (method = consensus)
6) Report read numbers through the pipeline
7) Write output
(23, 1078)


'qiime2/stats-dada2-asv.qzv'

## 6. Feature table and representative sequence summaries

Summarize ASV abundance and sequence content prior to decontamination. These summaries provide a baseline for evaluating how subsequent decontamination and filtering steps affect the dataset.

The ASV feature table summarizes how many reads are assigned to each ASV in each sample, while the representative sequence summary provides sequence-level information for each inferred ASV. Inspecting these outputs at this stage helps to:
- assess sequencing depth and feature richness across samples
- identify samples with unusually low read counts
- confirm that representative sequences have reasonable lengths after trimming and denoising
- establish a reference point for evaluating how later filtering steps change the dataset

**Output:** `qiime2/`
- ASV feature table summary: `table-asv.qzv`
- Representative sequences summary: `rep-seqs-asv.qzv`

In [9]:
from qiime2.plugins import feature_table

# Summarize ASV feature table (abundance per sample)
table_summary = feature_table.visualizers.summarize(
    table=dada2_results.table,
    sample_metadata=sample_metadata
)
table_summary.visualization.save("qiime2/table-asv.qzv")

# Summarize representative ASV sequences
seqs_summary = feature_table.visualizers.tabulate_seqs(data=dada2_results.representative_sequences)
seqs_summary.visualization.save("qiime2/rep-seqs-asv.qzv")

/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


'qiime2/rep-seqs-asv.qzv'

## 7. Decontamination using negative controls

Laboratory and reagent contaminants are common in amplicon sequencing experiments, especially in low-biomass samples. To address this, the prevalence-based method implemented in the QIIME 2 decontam plugin is used. This approach identifies features that occur more frequently in negative control samples than in true biological samples. This step is intentionally performed before low-abundance filtering so that negative controls are still present and informative. Removing low-abundance features first could eliminate contaminants before they are detected, reducing the effectiveness of decontamination.

Decontamination strategy
- Contaminant identification is based on prevalence in negative controls (role = negative_control).
- Features with decontam p-values ≤ 0.1 are considered likely contaminants and removed.
- Features with missing p-values are retained, resulting in a conservative filtering strategy that minimizes the risk of removing true biological taxa.

**Output:** `qiime2/`
- Decontaminated ASV feature table: `table-asv-decontam.qza` + `.qzv`
- Decontaminated representative sequences: `rep-seqs-asv-decontam.qza` + `.qzv`
- Decontamination statistics: `decontam-scores-asv.qza` + `.qzv`

In [10]:
from qiime2.plugins import quality_control

# Identify contaminants using prevalence-based decontam
decontam_scores = quality_control.methods.decontam_identify(
    table=dada2_results.table,
    metadata=sample_metadata,
    method="prevalence",
    prev_control_column="role",
    prev_control_indicator="negative_control"
)

# Save decontamination scores
decontam_scores.decontam_scores.save("qiime2/decontam-scores-asv.qza")

# Convert decontam scores to Metadata for filtering and visualization
decontam_scores_md = decontam_scores.decontam_scores.view(Metadata)

# Visualize decontamination statistics
decontam_viz = q2_metadata.visualizers.tabulate(input=decontam_scores_md)
decontam_viz.visualization.save("qiime2/decontam-scores-asv.qzv")

# Filter features based on decontam p-values
# Retain features with p > 0.1 or missing p-values
decontam_filtered = feature_table.methods.filter_features(
    table=dada2_results.table,
    metadata=decontam_scores_md,
    where="[p] > 0.1 OR [p] IS NULL"
)
decontam_filtered.filtered_table.save("qiime2/table-asv-decontam.qza")

# Report ASV counts before and after decontamination
n_before = dada2_results.table.view(pd.DataFrame).shape[1]
n_after = decontam_filtered.filtered_table.view(pd.DataFrame).shape[1]

print(f"ASVs before decontam: {n_before}")
print(f"ASVs after decontam:  {n_after}")

# Filter representative sequences to match the decontaminated table
decontam_seqs = feature_table.methods.filter_seqs(
    data=dada2_results.representative_sequences,
    table=decontam_filtered.filtered_table
)
decontam_seqs.filtered_data.save("qiime2/rep-seqs-asv-decontam.qza")

# Summarize decontaminated feature table
decontam_table_summary = feature_table.visualizers.summarize(
    table=decontam_filtered.filtered_table,
    sample_metadata=sample_metadata
)
decontam_table_summary.visualization.save("qiime2/table-asv-decontam.qzv")

# Summarize decontaminated representative sequences
decontam_seqs_summary = feature_table.visualizers.tabulate_seqs(data=decontam_seqs.filtered_data)
decontam_seqs_summary.visualization.save("qiime2/rep-seqs-asv-decontam.qzv")

We recommend 5 control samples - 1 found
Running external command line application. This may print messages to stdout and/or stderr.
The commands to be run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_decontam.R --asv_table_path /tmp/tmpvtfixbaq/temp_ASV_table.csv --threshold 0.1 --decon_method prevalence --output_track /tmp/tmpvtfixbaq/track.tsv --meta_table_path /tmp/tmpvtfixbaq/temp_metadata.csv --freq_con_column None --prev_control_or_exp_sample_column role --prev_control_sample_indicator negative_control

R version 4.3.3 (2024-02-29) 
[1] "Input ASV and Metadata files found"
Write output


/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/qiime2/metadata/metadata.py:610: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan na

ASVs before decontam: 1078
ASVs after decontam:  1077


/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


'qiime2/rep-seqs-asv-decontam.qzv'

## 8. Low-abundance filtering

Low-abundance features contribute little biological signal while increasing sparsity and multiple-testing burden. Performing this step after decontamination ensures contaminants are not removed prematurely.

Filtering criteria:
- Minimum total frequency ≥ 10 across all samples
- Present in at least 2 samples

**Output:** `qiime2/`
- Cleaned ASV table: `table-asv-decontam-filtered.qza` + `.qzv`
- Cleaned representative sequences: `rep-seqs-asv-decontam-filtered.qza` + `.qzv`

In [11]:
# Filter low-abundance features after decontamination
postfilt = feature_table.methods.filter_features(
    table=decontam_filtered.filtered_table,
    min_frequency=10,
    min_samples=2
)

n_after_filter = postfilt.filtered_table.view(pd.DataFrame).shape[1]
print(f"ASVs after abundance filtering: {n_after_filter}")

# Save filtered ASV table
postfilt.filtered_table.save("qiime2/table-asv-decontam-filtered.qza")

# Filter representative sequences to match filtered table
postfilt_seqs = feature_table.methods.filter_seqs(
    data=decontam_seqs.filtered_data,
    table=postfilt.filtered_table
)
postfilt_seqs.filtered_data.save("qiime2/rep-seqs-asv-decontam-filtered.qza")

# Summarize filtered ASV table
postfilt_summary = feature_table.visualizers.summarize(
    table=postfilt.filtered_table,
    sample_metadata=sample_metadata
)
postfilt_summary.visualization.save("qiime2/table-asv-decontam-filtered.qzv")

# Summarize filtered representative sequences
postfilt_seqs_viz = feature_table.visualizers.tabulate_seqs(data=postfilt_seqs.filtered_data)
postfilt_seqs_viz.visualization.save("qiime2/rep-seqs-asv-decontam-filtered.qzv")

# Report ASV counts before and after abundance filtering


ASVs after abundance filtering: 474


/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


'qiime2/rep-seqs-asv-decontam-filtered.qzv'

## 9. OTU clustering (97% de novo)

While ASVs provide single-nucleotide resolution and improved reproducibility, many published studies and reference datasets are still based on OTUs clustered at 97% sequence identity. Performing de novo OTU clustering allows direct comparison between ASV-based and OTU-based results using the same underlying sequencing data.

Clustering is performed after decontamination and low-abundance filtering, ensuring that OTUs are constructed from high-quality, biologically meaningful sequences rather than technical artifacts or contaminants.

Clustering strategy:
- De novo clustering: OTUs are formed based solely on sequence similarity within the dataset.
- Identity threshold: 97%, a commonly used approximation of species-level groupings in 16S rRNA gene studies.

**Output:** `qiime2/`
- OTU feature table: `table-otu.qza` + `.qzv`
- OTU representative sequences: `rep-seqs-otu.qza` + `.qzv`

In [12]:
from qiime2.plugins import vsearch

# De novo OTU clustering at 97% sequence identity
otu_results = vsearch.methods.cluster_features_de_novo(
    table=postfilt.filtered_table,
    sequences=postfilt_seqs.filtered_data,
    perc_identity=0.97
)

# Save OTU table and representative sequences
otu_results.clustered_table.save("qiime2/table-otu.qza")
otu_results.clustered_sequences.save("qiime2/rep-seqs-otu.qza")

# Summarize OTU feature table
otu_summary = feature_table.visualizers.summarize(
    table=otu_results.clustered_table,
    sample_metadata=sample_metadata
)
otu_summary.visualization.save("qiime2/table-otu.qzv")

# Summarize OTU representative sequences
otu_seqs_summary = feature_table.visualizers.tabulate_seqs(data=otu_results.clustered_sequences)
otu_seqs_summary.visualization.save("qiime2/rep-seqs-otu.qzv")

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --cluster_size /tmp/tmpmjrerhr5 --id 0.97 --centroids /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-0kcmone7 --uc /tmp/tmpoquulfum --strand plus --qmask none --xsize --threads 1 --minseqlength 1 --fasta_width 0



vsearch v2.22.1_linux_x86_64, 1007.3GB RAM, 80 cores
https://github.com/torognes/vsearch

Reading file /tmp/tmpmjrerhr5 100%
198039 nt in 474 seqs, min 386, max 428, avg 418
Sorting by abundance 100%
Counting k-mers 100%
Clustering 100%
Sorting clusters 100%
Writing clusters 100%
Clusters: 244 Size min 1, max 12, avg 1.9
Singletons: 136, 28.7% of seqs, 55.7% of clusters
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


'qiime2/rep-seqs-otu.qzv'

## 10. Taxonomy + barplots (ASVs & OTUs)

Taxonomic classification enables biological interpretation of sequence variants by mapping them to known bacterial lineages. A Naive Bayes classifier trained on the SILVA reference database is used to assign taxonomy to both ASVs and OTUs. Performing classification for both feature types allows direct comparison between high-resolution ASV-based results and legacy OTU-based analyses.

Stacked taxonomic barplots summarize the relative abundance of taxa across samples and metadata categories, providing an intuitive overview of community structure and dominant lineages.

**Output:** `qiime2/`
- Taxonomy artifacts and visualizations: 
  - ASVs: `taxonomy-asv.qza` + `.qzv`
  - OTUs: `qiime2/taxonomy-otu.qza` + `.qzv`
- Taxonomic barplots:
    - ASVs: `qiime2/taxa-barplots-asv.qzv`
    - OTUs: `qiime2/taxa-barplots-otu.qzv` 

In [13]:
from qiime2.plugins import feature_classifier, taxa

# Load pre-trained SILVA Naive Bayes classifier
silva_classifier = Artifact.load("ref/silva-138-99-nb-classifier.qza")

# Taxonomic classification of ASVs
asv_taxonomy = feature_classifier.methods.classify_sklearn(
    classifier=silva_classifier,
    reads=postfilt_seqs.filtered_data
)
asv_taxonomy.classification.save("qiime2/taxonomy-asv.qza")

# Tabulate ASV taxonomy
asv_tax_viz = q2_metadata.visualizers.tabulate(input=asv_taxonomy.classification.view(Metadata))
asv_tax_viz.visualization.save("qiime2/taxonomy-asv.qzv")

# Generate ASV taxonomic barplot
asv_barplot = taxa.visualizers.barplot(
    table=postfilt.filtered_table,
    taxonomy=asv_taxonomy.classification,
    metadata=sample_metadata
)
asv_barplot.visualization.save("qiime2/taxa-barplot-asv.qzv")

# Taxonomic classification of OTUs
otu_taxonomy = feature_classifier.methods.classify_sklearn(
    classifier=silva_classifier,
    reads=otu_results.clustered_sequences
)
otu_taxonomy.classification.save("qiime2/taxonomy-otu.qza")

# Tabulate OTU taxonomy
otu_tax_viz = q2_metadata.visualizers.tabulate(input=otu_taxonomy.classification.view(Metadata))
otu_tax_viz.visualization.save("qiime2/taxonomy-otu.qzv")

# Generate OTU taxonomic barplot
otu_barplot = taxa.visualizers.barplot(
    table=otu_results.clustered_table,
    taxonomy=otu_taxonomy.classification,
    metadata=sample_metadata
)
otu_barplot.visualization.save("qiime2/taxa-barplot-otu.qzv")

'qiime2/taxa-barplot-otu.qzv'

## 11. Phylogenetic trees (ASVs & OTUs)

Phylogenetic relationships between sequence variants are required for calculating diversity metrics that incorporate evolutionary information, such as Faith’s Phylogenetic Diversity and UniFrac distances. Representative sequences are first aligned using MAFFT, a multiple sequence alignment algorithm optimized for large datasets. Highly variable or poorly aligned regions are then masked to reduce noise. Phylogenetic trees are subsequently inferred using FastTree, which efficiently estimates approximately maximum-likelihood trees.

Rooted trees are required for Faith’s PD and UniFrac calculations, while unrooted trees are retained for completeness and potential downstream use.

**Output:** `qiime2/`
- Aligned representative sequences:
  - ASVs: `aligned-rep-seqs-asv.qza`
  - OTUs: `aligned-rep-seqs-otu.qza`
- Masked aligned sequences:
  - ASVs: `masked-aligned-rep-seqs-asv.qza`
  - OTUs: `masked-aligned-rep-seqs-otu.qza`
- Unrooted phylogenetic trees:
  - ASVs: `unrooted-tree-asv.qza`
  - OTUs: `unrooted-tree-otu.qza`
- Rooted phylogenetic trees:
  - ASVs: `rooted-tree-asv.qza`
  - OTUs: `rooted-tree-otu.qza`

In [14]:
from qiime2.plugins import phylogeny

# Build phylogenetic tree for ASVs
asv_tree = phylogeny.pipelines.align_to_tree_mafft_fasttree(sequences=postfilt_seqs.filtered_data)
asv_tree.alignment.save("qiime2/aligned-rep-seqs-asv.qza")
asv_tree.masked_alignment.save("qiime2/masked-aligned-rep-seqs-asv.qza")
asv_tree.tree.save("qiime2/unrooted-tree-asv.qza")
asv_tree.rooted_tree.save("qiime2/rooted-tree-asv.qza")

# Build phylogenetic tree for OTUs
otu_tree = phylogeny.pipelines.align_to_tree_mafft_fasttree(sequences=otu_results.clustered_sequences)
otu_tree.alignment.save("qiime2/aligned-rep-seqs-otu.qza")
otu_tree.masked_alignment.save("qiime2/masked-aligned-rep-seqs-otu.qza")
otu_tree.tree.save("qiime2/unrooted-tree-otu.qza")
otu_tree.rooted_tree.save("qiime2/rooted-tree-otu.qza")

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 1 /tmp/qiime2/211933/data/8a95ad4a-4d89-4570-bebc-8553922d287c/data/dna-sequences.fasta



inputfile = orig
474 x 428 - 386 d
nthread = 1
nthreadpair = 1
nthreadtb = 1
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
  401 / 474 (thread    0)
done.

Constructing a UPGMA tree (efffree=0) ... 
  470 / 474
done.

Progressive alignment 1/2... 
STEP   398 / 473 (thread    0)
Reallocating..done. *alloclen = 1862
STEP   473 / 473 (thread    0) h
done.

Making a distance matrix from msa.. 
  400 / 474 (thread    0)
done.

Constructing a UPGMA tree (efffree=1) ... 
  470 / 474
done.

Progressive alignment 2/2... 
STEP   447 / 473 (thread    0)
Reallocating..done. *alloclen = 1863
STEP   473 / 473 (thread    0) h
done.

disttbfast (nuc) Version 7.526
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
1 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto 

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTree -quote -nt /tmp/qiime2/211933/data/48f64dfd-b91e-4e09-ae0b-eb6ed320f8c1/data/aligned-dna-sequences.fasta



FastTree Version 2.1.11 Double precision (No SSE3)
Alignment: /tmp/qiime2/211933/data/48f64dfd-b91e-4e09-ae0b-eb6ed320f8c1/data/aligned-dna-sequences.fasta
Nucleotide distances: Jukes-Cantor Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Jukes-Cantor, CAT approximation with 20 rate categories
      0.10 seconds: Joined    100 of    427
      0.24 seconds: Joined    300 of    427
Initial topology in 0.31 seconds
Refining topology: 35 rounds ME-NNIs, 2 rounds ME-SPRs, 17 rounds ML-NNIs
      0.34 seconds: ME NNI round 2 of 35, 401 of 428 splits, 31 changes (max delta 0.029)
      0.45 seconds: SPR round   1 of   2, 101 of 858 nodes
      0.63 seconds: SPR round   1 of   2, 301 of 858 nodes
      0.77 seconds: SPR round   1 of   2, 501 of 858 nodes
      0.91 seconds: SPR round   1 of   2, 701 of 858 nodes
      1.05 seconds: ME NNI round 12 of 35, 1 of 428 splits
      1.22 sec

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 1 /tmp/qiime2/211933/data/1cb91594-8aba-450c-a4cf-30b26c299b9a/data/dna-sequences.fasta



inputfile = orig
244 x 428 - 386 d
nthread = 1
nthreadpair = 1
nthreadtb = 1
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
  201 / 244 (thread    0)
done.

Constructing a UPGMA tree (efffree=0) ... 
  240 / 244
done.

Progressive alignment 1/2... 
STEP   175 / 243 (thread    0)
Reallocating..done. *alloclen = 1862
STEP   243 / 243 (thread    0)
done.

Making a distance matrix from msa.. 
  200 / 244 (thread    0)
done.

Constructing a UPGMA tree (efffree=1) ... 
  240 / 244
done.

Progressive alignment 2/2... 
STEP   197 / 243 (thread    0)
Reallocating..done. *alloclen = 1862
STEP   243 / 243 (thread    0)
done.

disttbfast (nuc) Version 7.526
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
1 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto inpu

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTree -quote -nt /tmp/qiime2/211933/data/d26ff304-2ee7-4f8b-ae3a-322e92350133/data/aligned-dna-sequences.fasta



FastTree Version 2.1.11 Double precision (No SSE3)
Alignment: /tmp/qiime2/211933/data/d26ff304-2ee7-4f8b-ae3a-322e92350133/data/aligned-dna-sequences.fasta
Nucleotide distances: Jukes-Cantor Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Jukes-Cantor, CAT approximation with 20 rate categories
      0.11 seconds: Joined    200 of    241
Initial topology in 0.13 seconds
Refining topology: 32 rounds ME-NNIs, 2 rounds ME-SPRs, 16 rounds ML-NNIs
      0.24 seconds: SPR round   1 of   2, 101 of 486 nodes
      0.34 seconds: SPR round   1 of   2, 201 of 486 nodes
      0.51 seconds: SPR round   1 of   2, 401 of 486 nodes
      0.62 seconds: ME NNI round 12 of 32, 201 of 242 splits, 3 changes (max delta 0.005)
      0.79 seconds: SPR round   2 of   2, 201 of 486 nodes
      0.97 seconds: SPR round   2 of   2, 401 of 486 nodes
Total branch-length 12.367 after 1.08 sec
      1.07 secon

'qiime2/rooted-tree-otu.qza'

## 12. Remove controls for diversity analyses

Negative controls are removed from the ASV and OTU feature tables to ensure that diversity analyses only include biological samples. This step is crucial for accurate alpha and beta diversity calculations. Including controls would artificially inflate diversity estimates and distort distance-based comparisons between biological samples.

Sample removal is performed using metadata-based filtering, retaining only samples annotated as biological in the role column of the metadata.

**Output:** `qiime2/`
- ASV feature table without controls: `table-asv-nocontrols.qza` + `.qzv`
- OTU feature table without controls: `table-otu-nocontrols.qza` + `.qzv`

In [15]:
# Remove negative controls from ASV table
asv_nocontrols = feature_table.methods.filter_samples(
    table=postfilt.filtered_table,
    metadata=sample_metadata,
    where="role='biological'"
)
asv_nocontrols.filtered_table.save("qiime2/table-asv-nocontrols.qza")

asv_bio_summary = feature_table.visualizers.summarize(
    table=asv_nocontrols.filtered_table,
    sample_metadata=sample_metadata
)
asv_bio_summary.visualization.save("qiime2/table-asv-nocontrols.qzv")

# Remove negative controls from OTU table
otu_nocontrols = feature_table.methods.filter_samples(
    table=otu_results.clustered_table,
    metadata=sample_metadata,
    where="role='biological'"
)
otu_nocontrols.filtered_table.save("qiime2/table-otu-nocontrols.qza")

otu_bio_summary = feature_table.visualizers.summarize(
    table=otu_nocontrols.filtered_table,
    sample_metadata=sample_metadata
)
otu_bio_summary.visualization.save("qiime2/table-otu-nocontrols.qzv")

/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning

'qiime2/table-otu-nocontrols.qzv'

## 13. Core diversity metrics (ASVs & OTUs)

**Purpose:** Quantify within-sample (alpha) and between-sample (beta) diversity for biological samples using both non-phylogenetic and phylogeny-aware metrics.

Diversity analyses provide insight into microbial community structure and how it varies across samples and experimental groups. Two complementary classes of metrics are calculated:
- Non-phylogenetic metrics describe diversity based solely on observed features and their abundances.
- Phylogeny-aware metrics incorporate evolutionary relationships between taxa, using the rooted phylogenetic trees constructed previously.

To ensure fair comparisons across samples with unequal sequencing depth, feature tables are rarefied to a uniform sampling depth prior to diversity calculation. The chosen depth represents a compromise between retaining as many samples as possible and preserving sufficient sequencing depth per sample.

**Alpha diversity metrics:**
- Shannon index: captures both richness and evenness; higher values indicate more diverse and more evenly distributed communities.
- Faith’s Phylogenetic Diversity (PD): measures total branch length spanned by taxa observed in a sample, reflecting phylogenetic richness.

**Beta diversity metrics:**
- Bray–Curtis dissimilarity: abundance-based, sensitive to changes in relative abundance of shared taxa.
- Weighted UniFrac: phylogenetic and abundance-weighted, emphasizing shifts in dominant lineages.
- Unweighted UniFrac: phylogenetic and presence/absence-based, emphasizing differences in community membership, including rare taxa.

Ordination of beta diversity distance matrices is visualized using Emperor plots, enabling exploration of sample clustering patterns in reduced dimensional space.

**Output:** `qiime2/`
- Alpha diversity vectors:
    - ASVs (`diversity-asv/`): `shannon-asv.qza`, `faith-pd-asv.qza`
    - OTUs (`diversity-otu/`): `shannon-otu.qza`, `faith-pd-otu.qza`
- Beta diversity distance matrices:
    - ASVs (`diversity-asv/`): `bray-curtis-asv.qza`, `weighted-unifrac-asv.qza`, `unweighted-unifrac-asv.qza`
    - OTUs (`diversity-otu/`): `bray-curtis-otu.qza`, `weighted-unifrac-otu.qza`, `unweighted-unifrac-otu.qza`
- Emperor visualizations:
    - ASVs (`diversity-asv/`): `bray-curtis-emperor-asv.qzv`, `weighted-unifrac-emperor-asv.qzv`, `unweighted-unifrac-emperor-asv.qzv`
    - OTUs (`diversity-otu/`): `bray-curtis-emperor-otu.qzv`, `weighted-unifrac-emperor-otu.qzv`, `unweighted-unifrac-emperor-otu.qzv`

In [16]:
from qiime2.plugins import diversity

# SAMPLING_DEPTH was chosen after inspecting the per-sample frequency distribution in qiime2/table-asv-nocontrols.qzv 
# and qiime2/table-otu-nocontrols.qzv, balancing sample retention and depth.
SAMPLING_DEPTH = 3000

# ASV diversity metrics
asv_div = diversity.pipelines.core_metrics_phylogenetic(
    phylogeny=asv_tree.rooted_tree,
    table=asv_nocontrols.filtered_table,
    sampling_depth=SAMPLING_DEPTH,
    metadata=sample_metadata
)

# Alpha diversity
asv_div.shannon_vector.save("qiime2/diversity-asv/shannon_asv.qza")
asv_div.faith_pd_vector.save("qiime2/diversity-asv/faith_pd_asv.qza")

# Beta diversity
asv_div.bray_curtis_distance_matrix.save("qiime2/diversity-asv/bray_curtis_asv.qza")
asv_div.bray_curtis_emperor.save("qiime2/diversity-asv/bray_curtis_emperor_asv.qzv")

asv_div.unweighted_unifrac_distance_matrix.save("qiime2/diversity-asv/unweighted_unifrac_asv.qza")
asv_div.unweighted_unifrac_emperor.save("qiime2/diversity-asv/unweighted_unifrac_emperor_asv.qzv")

asv_div.weighted_unifrac_distance_matrix.save("qiime2/diversity-asv/weighted_unifrac_asv.qza")
asv_div.weighted_unifrac_emperor.save("qiime2/diversity-asv/weighted_unifrac_emperor_asv.qzv")

# OTU diversity metrics
otu_div = diversity.pipelines.core_metrics_phylogenetic(
    phylogeny=otu_tree.rooted_tree,
    table=otu_nocontrols.filtered_table,
    sampling_depth=SAMPLING_DEPTH,
    metadata=sample_metadata
)

# Alpha diversity
otu_div.shannon_vector.save("qiime2/diversity-otu/shannon_otu.qza")
otu_div.faith_pd_vector.save("qiime2/diversity-otu/faith_pd_otu.qza")

# Beta diversity
otu_div.bray_curtis_distance_matrix.save("qiime2/diversity-otu/bray_curtis_otu.qza")
otu_div.bray_curtis_emperor.save("qiime2/diversity-otu/bray_curtis_emperor_otu.qzv")

otu_div.unweighted_unifrac_distance_matrix.save("qiime2/diversity-otu/unweighted_unifrac_otu.qza")
otu_div.unweighted_unifrac_emperor.save("qiime2/diversity-otu/unweighted_unifrac_emperor_otu.qzv")

otu_div.weighted_unifrac_distance_matrix.save("qiime2/diversity-otu/weighted_unifrac_otu.qza")
otu_div.weighted_unifrac_emperor.save("qiime2/diversity-otu/weighted_unifrac_emperor_otu.qzv")

/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/skbio/util/_warning.py:60: UserWarning: observed_otus is deprecated as of 0.6.0.
  warn(f"{func.__name__} is deprecated as of {ver}.")
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:146: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.013347827709485815 and the largest is 1.9423910143599137.
  warn(


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-e24dkrxv -t /tmp/qiime2/211933/data/296ad319-2ca4-4857-b782-071a96dcf6d0/data/tree.nwk -o /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-hufukz06

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

ssu -i /tmp/qiime2/211933/data/a516c24c-a4f1-4d35-b7ca-815053560956/data/feature-table.biom -t /tmp/qiime2/211933/data/296ad319-2ca4-4857-b782-071a96dcf6d0/data/tree.nwk -m unweighted -o /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-1dntvia5

Running ex

/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:146: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.005358539775203256 and the largest is 1.028096466365366.
  warn(
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:146: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-5uxg79a8 -t /tmp/qiime2/211933/data/9559c13d-10e1-458a-884c-702d1127d381/data/tree.nwk -o /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-jba4fwgd

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

ssu -i /tmp/qiime2/211933/data/038909af-f7a3-4186-9acd-ef5d1cf0ff9a/data/feature-table.biom -t /tmp/qiime2/211933/data/9559c13d-10e1-458a-884c-702d1127d381/data/tree.nwk -m unweighted -o /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-24wo2ehw

Running ex

/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:146: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.005026017587794833 and the largest is 1.0397748842540648.
  warn(
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:146: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes sectio

'qiime2/diversity-otu/weighted_unifrac_emperor_otu.qzv'

## 14.  Alpha rarefaction (ASVs & OTUs)

Alpha rarefaction evaluates whether sequencing depth is sufficient to capture within-sample diversity. Rarefaction curves visualize alpha diversity (e.g., observed features and Shannon) as a function of sampling depth. Curves that plateau suggest adequate depth, whereas strongly increasing curves suggest undersampling. 

**Output:** `qiime2/`
- ASV rarefaction curves: `alpha-rarefaction-asv.qzv`
- OTU rarefaction curves: `alpha-rarefaction-otu.qzv`

In [17]:
# Set maximum depth based on feature table summaries
MAX_DEPTH = 7000

# Alpha rarefaction for ASVs
asv_rarefaction = diversity.visualizers.alpha_rarefaction(
    table=asv_nocontrols.filtered_table,
    phylogeny=asv_tree.rooted_tree,
    max_depth=MAX_DEPTH,
    metadata=sample_metadata
)
asv_rarefaction.visualization.save("qiime2/alpha-rarefaction-asv.qzv")

# Alpha rarefaction for OTUs
otu_rarefaction = diversity.visualizers.alpha_rarefaction(
    table=otu_nocontrols.filtered_table,
    phylogeny=otu_tree.rooted_tree,
    max_depth=MAX_DEPTH,
    metadata=sample_metadata
)
otu_rarefaction.visualization.save("qiime2/alpha-rarefaction-otu.qzv")

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-86z32ff_ -t /tmp/qiime2/211933/data/d15c1256-7022-4454-bc53-889d796ee386/data/tree.nwk -o /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-zvn5aqy3



/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-7low38ko -t /tmp/qiime2/211933/data/d15c1256-7022-4454-bc53-889d796ee386/data/tree.nwk -o /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-5b0nldif



/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-lg60y_rq -t /tmp/qiime2/211933/data/d15c1256-7022-4454-bc53-889d796ee386/data/tree.nwk -o /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-f64s2dtb



...

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-uollmqen -t /tmp/qiime2/211933/data/b37f0b63-a1ab-423a-a3f7-a264dea2105b/data/tree.nwk -o /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-a45yqz8x



/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-zcs3gi4_ -t /tmp/qiime2/211933/data/b37f0b63-a1ab-423a-a3f7-a264dea2105b/data/tree.nwk -o /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-i62o78hi



/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df

'qiime2/alpha-rarefaction-otu.qzv'

## 15. Group significance

Group significance testing evaluates whether observed differences in alpha and beta diversity metrics are associated with experimental variables rather than random variation.

Two complementary approaches are used:

**Alpha diversity significance:** Alpha diversity metrics (Shannon index and Faith’s Phylogenetic Diversity) are compared across groups using non-parametric statistical tests as implemented in QIIME 2. These tests assess whether groups differ in within-sample diversity, accounting for the non-normal distribution typical of microbiome data.

**Beta diversity significance:** Beta diversity distance matrices (Bray–Curtis, weighted UniFrac, and unweighted UniFrac) are tested using PERMANOVA with pairwise comparisons. PERMANOVA evaluates whether community composition differs significantly between groups in multivariate space.

Using both non-phylogenetic and phylogenetic distance metrics allows detection of:
- abundance-driven shifts (Bray–Curtis),
- changes in dominant lineages (weighted UniFrac),
- changes in community membership, including rare taxa (unweighted UniFrac).

Tests are performed for the following grouping variables:
- Disease state
- Sample type

**Output:**

**Alpha diversity group significance**
- ASVs: `qiime2/diversity-asv/`
    - Shannon index: `shannon-group-significance.qzv`
    - Faith’s PD: `faith-pd-group-significance.qzv`
- OTUs: `qiime2/diversity-otu/`
    - Shannon index: `shannon-group-significance.qzv`
    - Faith’s PD: `faith-pd-group-significance.qzv`

**Beta diversity group significance**
- ASVs: `qiime2/diversity-asv/`
    - Bray–Curtis
        - Disease state: `bray-disease-state-significance.qzv`
        - Sample type: `bray-type-significance.qzv`
    - Weighted UniFrac
        - Disease state: `weighted-unifrac-disease-state-significance.qzv`
        - Sample type: `weighted-unifrac-type-significance.qzv`
    - Unweighted UniFrac
        - Disease state: `unweighted-unifrac-disease-state-significance.qzv`
        - Sample type: `unweighted-unifrac-type-significance.qzv`
- OTUs: `qiime2/diversity-otu/`
    - Bray–Curtis
        - Disease state: `bray-disease-state-significance.qzv`
        - Sample type: `bray-type-significance.qzv`
    - Weighted UniFrac
        - Disease state: `weighted-unifrac-disease-state-significance.qzv`
        - Sample type: `weighted-unifrac-type-significance.qzv`
    - Unweighted UniFrac
        - Disease state: `unweighted-unifrac-disease-state-significance.qzv`
        - Sample type: `unweighted-unifrac-type-significance.qzv`

In [18]:
# ASV-based group significance testing
# Alpha diversity: Shannon index
asv_shannon_sig = diversity.visualizers.alpha_group_significance(
    alpha_diversity=asv_div.shannon_vector,
    metadata=sample_metadata
)
asv_shannon_sig.visualization.save("qiime2/diversity-asv/shannon-group-significance.qzv")

# Alpha diversity: Faith's PD
asv_faith_sig = diversity.visualizers.alpha_group_significance(
    alpha_diversity=asv_div.faith_pd_vector,
    metadata=sample_metadata
)
asv_faith_sig.visualization.save("qiime2/diversity-asv/faith-pd-group-significance.qzv")

# Beta diversity: Bray–Curtis
asv_bray_disease = diversity.visualizers.beta_group_significance(
    distance_matrix=asv_div.bray_curtis_distance_matrix,
    metadata=sample_metadata.get_column("disease-state"),
    pairwise=True
)
asv_bray_disease.visualization.save("qiime2/diversity-asv/bray-disease-state-significance.qzv")

asv_bray_type = diversity.visualizers.beta_group_significance(
    distance_matrix=asv_div.bray_curtis_distance_matrix,
    metadata=sample_metadata.get_column("type"),
    pairwise=True
)
asv_bray_type.visualization.save("qiime2/diversity-asv/bray-type-significance.qzv")

# Beta diversity: Weighted UniFrac
asv_wu_disease = diversity.visualizers.beta_group_significance(
    distance_matrix=asv_div.weighted_unifrac_distance_matrix,
    metadata=sample_metadata.get_column("disease-state"),
    pairwise=True
)
asv_wu_disease.visualization.save("qiime2/diversity-asv/weighted-unifrac-disease-state-significance.qzv")

asv_wu_type = diversity.visualizers.beta_group_significance(
    distance_matrix=asv_div.weighted_unifrac_distance_matrix,
    metadata=sample_metadata.get_column("type"),
    pairwise=True
)
asv_wu_type.visualization.save("qiime2/diversity-asv/weighted-unifrac-type-significance.qzv")

# Beta diversity: Unweighted UniFrac
asv_uu_disease = diversity.visualizers.beta_group_significance(
    distance_matrix=asv_div.unweighted_unifrac_distance_matrix,
    metadata=sample_metadata.get_column("disease-state"),
    pairwise=True
)
asv_uu_disease.visualization.save("qiime2/diversity-asv/unweighted-unifrac-disease-state-significance.qzv")

asv_uu_type = diversity.visualizers.beta_group_significance(
    distance_matrix=asv_div.unweighted_unifrac_distance_matrix,
    metadata=sample_metadata.get_column("type"),
    pairwise=True
)
asv_uu_type.visualization.save("qiime2/diversity-asv/unweighted-unifrac-type-significance.qzv")

# OTU-based group significance testing
# Alpha diversity: Shannon index
otu_shannon_sig = diversity.visualizers.alpha_group_significance(
    alpha_diversity=otu_div.shannon_vector,
    metadata=sample_metadata
)
otu_shannon_sig.visualization.save("qiime2/diversity-otu/shannon-group-significance.qzv")

# Alpha diversity: Faith's PD
otu_faith_sig = diversity.visualizers.alpha_group_significance(
    alpha_diversity=otu_div.faith_pd_vector,
    metadata=sample_metadata
)
otu_faith_sig.visualization.save("qiime2/diversity-otu/faith-pd-group-significance.qzv")

# Beta diversity: Bray–Curtis
otu_bray_disease = diversity.visualizers.beta_group_significance(
    distance_matrix=otu_div.bray_curtis_distance_matrix,
    metadata=sample_metadata.get_column("disease-state"),
    pairwise=True
)
otu_bray_disease.visualization.save("qiime2/diversity-otu/bray-disease-state-significance.qzv")

otu_bray_type = diversity.visualizers.beta_group_significance(
    distance_matrix=otu_div.bray_curtis_distance_matrix,
    metadata=sample_metadata.get_column("type"),
    pairwise=True
)
otu_bray_type.visualization.save("qiime2/diversity-otu/bray-type-significance.qzv")

# Beta diversity: Weighted UniFrac
otu_wu_disease = diversity.visualizers.beta_group_significance(
    distance_matrix=otu_div.weighted_unifrac_distance_matrix,
    metadata=sample_metadata.get_column("disease-state"),
    pairwise=True
)
otu_wu_disease.visualization.save("qiime2/diversity-otu/weighted-unifrac-disease-state-significance.qzv")

otu_wu_type = diversity.visualizers.beta_group_significance(
    distance_matrix=otu_div.weighted_unifrac_distance_matrix,
    metadata=sample_metadata.get_column("type"),
    pairwise=True
)
otu_wu_type.visualization.save("qiime2/diversity-otu/weighted-unifrac-type-significance.qzv")

# Beta diversity: Unweighted UniFrac
otu_uu_disease = diversity.visualizers.beta_group_significance(
    distance_matrix=otu_div.unweighted_unifrac_distance_matrix,
    metadata=sample_metadata.get_column("disease-state"),
    pairwise=True
)
otu_uu_disease.visualization.save("qiime2/diversity-otu/unweighted-unifrac-disease-state-significance.qzv")

otu_uu_type = diversity.visualizers.beta_group_significance(
    distance_matrix=otu_div.unweighted_unifrac_distance_matrix,
    metadata=sample_metadata.get_column("type"),
    pairwise=True
)
otu_uu_type.visualization.save("qiime2/diversity-otu/unweighted-unifrac-type-significance.qzv")

/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_diversity/_beta/_visualizer.py:174: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determi

'qiime2/diversity-otu/unweighted-unifrac-type-significance.qzv'

## 16. Identify unassigned / unclassified (ASVs & OTUs)

Inspect taxa that are poorly classified (e.g., Unassigned) or lack genus-level resolution. This can help diagnose reference database limitations, primer bias, low-quality reads, or unusual taxa, and it can guide decisions about filtering or reporting.

After taxonomic classification (here: SILVA-based), some features may remain:
- Unassigned: no confident match at the requested rank(s), often due to limited reference coverage or short/low-quality sequences
- Unclassified at genus: taxonomy string includes a genus placeholder (e.g., g__) with no genus name assigned

**!!Issue!!:** --> ValueError: All features were filtered, resulting in an empty table.

**Outputs:** `qiime2/`
- ASVs:
    - `table-asv-unassigned.qza` + `.qzv`
    - `table-asv-genus-unclassified.qza` + `.qzv`
- OTUs:
    - `table-otu-unassigned.qza` + `.qzv`
    - `table-otu-genus-unclassified.qza` + `.qzv`

In [19]:
def safe_taxa_filter_and_summarize(*, table, taxonomy, include, mode="exact", out_prefix):
    """
    Filter a table by taxonomy criteria and write outputs.
    If result is empty, skip gracefully (no crash).
    """
    try:
        res = taxa.methods.filter_table(
            table=table,
            taxonomy=taxonomy,
            include=include,
            mode=mode,
        )
        res.filtered_table.save(f"qiime2/{out_prefix}.qza")

        feature_table.visualizers.summarize(
            table=res.filtered_table,
            sample_metadata=sample_metadata,
        ).visualization.save(f"qiime2/{out_prefix}.qzv")

        print(f"Saved: qiime2/{out_prefix}.qza + .qzv")
        return res
    except ValueError as e:
        print(f"Skipping {out_prefix} (empty result): {e}")
        return None

# ASVs
safe_taxa_filter_and_summarize(
    table=asv_nocontrols.filtered_table,
    taxonomy=asv_taxonomy.classification,
    include="Unassigned",
    out_prefix="table-asv-unassigned",
)

safe_taxa_filter_and_summarize(
    table=asv_nocontrols.filtered_table,
    taxonomy=asv_taxonomy.classification,
    include="g__",
    mode="contains",
    out_prefix="table-asv-genus-unclassified",
)

# OTUs
safe_taxa_filter_and_summarize(
    table=otu_nocontrols.filtered_table,
    taxonomy=otu_taxonomy.classification,
    include="Unassigned",
    out_prefix="table-otu-unassigned",
)

safe_taxa_filter_and_summarize(
    table=otu_nocontrols.filtered_table,
    taxonomy=otu_taxonomy.classification,
    include="g__",
    mode="contains",
    out_prefix="table-otu-genus-unclassified",
)

Skipping table-asv-unassigned (empty result): All features were filtered, resulting in an empty table.


/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


Saved: qiime2/table-asv-genus-unclassified.qza + .qzv
Skipping table-otu-unassigned (empty result): All features were filtered, resulting in an empty table.


/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


Saved: qiime2/table-otu-genus-unclassified.qza + .qzv


Results (name = value)
-----------------------------------------------------------------------------------------------
filtered_table = <artifact: FeatureTable[Frequency] uuid: 31e6dde4-69e1-46bc-9ff9-f40e5947ce6a>

## 17. Differential abundance (ANCOM-BC) + genus collapse

**Purpose:** Identify taxa whose abundances differ significantly between disease states.

Microbiome feature tables contain count data that are compositional (only relative abundances are observed) and often show strong library size variation across samples. ANCOM-BC (Analysis of Composition of Microbiomes with Bias Correction) is designed for differential abundance testing under these conditions. It estimates bias-corrected log-fold changes and controls false discoveries, improving interpretability.

To capture patterns at different taxonomic resolutions, the analysis is performed:
- At the ASV level (highest resolution)
- At the OTU level (legacy-style resolution)
- At the genus level (collapsed tables) to reduce sparsity and improve interpretability

The fixed effect tested here is: *Disease state*

**Outputs:** `qiime2/`
- Differential abundance artifacts (*.qza)`
    - ASVs: 
        - `asv_disease-ancombc-differentials.qza`
        - `asv_genus_disease-ancombc-differentials.qza`     
    - OTUs:   
        - `otu_disease-ancombc-differentials.qza`
        - `otu_genus_disease-ancombc-differentials.qza`
- Differential abundance barplots (*.qzv)
    - ASVs: 
        - `asv_disease-ancombc-barplot.qzv`
        - `asv_genus_disease-ancombc-barplot.qzv`
    - OTUs: 
        - `otu_disease-ancombc-barplot.qzv`
        - `otu_genus_disease-ancombc-barplot.qzv`

In [20]:
import pandas as pd
from qiime2.plugins.composition.methods import ancombc
from qiime2.plugins.composition.visualizers import da_barplot

# Metadata cleanup for formula use
meta_df = sample_metadata.to_dataframe().copy()

if "disease-state" not in meta_df.columns:
    raise KeyError("Expected column 'disease-state' in sample metadata.")

# Make formula-safe name for ANCOM-BC
meta_df.rename(columns={"disease-state": "disease_state"}, inplace=True)
clean_metadata = Metadata(meta_df)

# Helper: subset metadata to table sample IDs
def subset_metadata_to_table(table_artifact, metadata: Metadata) -> Metadata:
    sample_ids = table_artifact.view(pd.DataFrame).index
    md = metadata.to_dataframe().loc[sample_ids].copy()
    md.index.name = "sample-id"
    return Metadata(md)

# Helper: run ANCOM-BC + save outputs
def run_ancombc_on_table(table_artifact, metadata: Metadata, output_prefix: str):
    print(f"\nRunning ANCOM-BC: {output_prefix}")

    md_sub = subset_metadata_to_table(table_artifact, metadata)

    # ANCOM-BC expects raw counts (NOT rarefied)
    differentials, = ancombc(
        table=table_artifact,
        metadata=md_sub,
        formula="disease_state",
    )

    Path("qiime2").mkdir(exist_ok=True)

    differentials_fp = f"qiime2/{output_prefix}-ancombc-differentials.qza"
    barplot_fp = f"qiime2/{output_prefix}-ancombc-barplot.qzv"

    differentials.save(differentials_fp)

    viz, = da_barplot(
        data=differentials,
        significance_threshold=0.05,
        effect_size_threshold=1.0,
    )
    viz.save(barplot_fp)

    return differentials

# ANCOM-BC at ASV and OTU resolution
asv_diff = run_ancombc_on_table(
    table_artifact=asv_nocontrols.filtered_table,
    metadata=clean_metadata,
    output_prefix="asv_disease",
)

otu_diff = run_ancombc_on_table(
    table_artifact=otu_nocontrols.filtered_table,
    metadata=clean_metadata,
    output_prefix="otu_disease",
)

# Genus collapse + ANCOM-BC at genus resolution
asv_genus = taxa.methods.collapse(
    table=asv_nocontrols.filtered_table,
    taxonomy=asv_taxonomy.classification,
    level=6,
).collapsed_table

otu_genus = taxa.methods.collapse(
    table=otu_nocontrols.filtered_table,
    taxonomy=otu_taxonomy.classification,
    level=6,
).collapsed_table

asv_genus_diff = run_ancombc_on_table(
    table_artifact=asv_genus,
    metadata=clean_metadata,
    output_prefix="asv_genus_disease",
)

otu_genus_diff = run_ancombc_on_table(
    table_artifact=otu_genus,
    metadata=clean_metadata,
    output_prefix="otu_genus_disease",
)


Running ANCOM-BC: asv_disease
Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_ancombc.R --inp_abundances_path /tmp/tmp_6b5s5bo/input.biom.tsv --inp_metadata_path /tmp/tmp_6b5s5bo/input.map.txt --md_column_types {"horse": "categorical", "role": "categorical", "type": "categorical", "tooth-number": "categorical", "tooth-location": "categorical", "replicate": "categorical", "gender": "categorical", "age": "categorical", "disease_state": "categorical", "DIN": "categorical"} --formula disease_state --p_adj_method holm --prv_cut 0.1 --lib_cut 0 --reference_levels ['disease_state::diseased'] --tol 1e-05 --max_iter 100 --conserve False --alpha 0.05 --output_loaf /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-mo0cty6_



/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_composition/_ancombc.py:64: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  term_alpha_value = (metadata.get_column(term)
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘jsonlite’

The following object is masked from ‘package:purrr’:

    fl

R version 4.3.3 (2024-02-29) 


New names:
• `` -> `...1`
'ancombc' has been fully evolved to 'ancombc2'. 
Explore the enhanced capabilities of our refined method!
Loading required package: foreach

Attaching package: ‘foreach’

The following objects are masked from ‘package:purrr’:

    accumulate, when

Loading required package: rngtools



Running ANCOM-BC: otu_disease
Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_ancombc.R --inp_abundances_path /tmp/tmp30oqye4e/input.biom.tsv --inp_metadata_path /tmp/tmp30oqye4e/input.map.txt --md_column_types {"horse": "categorical", "role": "categorical", "type": "categorical", "tooth-number": "categorical", "tooth-location": "categorical", "replicate": "categorical", "gender": "categorical", "age": "categorical", "disease_state": "categorical", "DIN": "categorical"} --formula disease_state --p_adj_method holm --prv_cut 0.1 --lib_cut 0 --reference_levels ['disease_state::diseased'] --tol 1e-05 --max_iter 100 --conserve False --alpha 0.05 --output_loaf /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-ddincyr_



...

New names:
• `` -> `...1`
'ancombc' has been fully evolved to 'ancombc2'. 
Explore the enhanced capabilities of our refined method!
Loading required package: foreach

Attaching package: ‘foreach’

The following objects are masked from ‘package:purrr’:

    accumulate, when

Loading required package: rngtools



Running ANCOM-BC: otu_genus_disease
Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_ancombc.R --inp_abundances_path /tmp/tmpsotzc9nu/input.biom.tsv --inp_metadata_path /tmp/tmpsotzc9nu/input.map.txt --md_column_types {"horse": "categorical", "role": "categorical", "type": "categorical", "tooth-number": "categorical", "tooth-location": "categorical", "replicate": "categorical", "gender": "categorical", "age": "categorical", "disease_state": "categorical", "DIN": "categorical"} --formula disease_state --p_adj_method holm --prv_cut 0.1 --lib_cut 0 --reference_levels ['disease_state::diseased'] --tol 1e-05 --max_iter 100 --conserve False --alpha 0.05 --output_loaf /tmp/qiime2/211933/processes/972472-1765707600.12@211933/tmp/q2-OutPath-3xdgfwwo



/apps/anaconda3/envs/qiime2-amplicon-2025.7/lib/python3.10/site-packages/q2_composition/_ancombc.py:64: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  term_alpha_value = (metadata.get_column(term)
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘jsonlite’

The following object is masked from ‘package:purrr’:

    fl

R version 4.3.3 (2024-02-29) 


New names:
• `` -> `...1`
'ancombc' has been fully evolved to 'ancombc2'. 
Explore the enhanced capabilities of our refined method!
Loading required package: foreach

Attaching package: ‘foreach’

The following objects are masked from ‘package:purrr’:

    accumulate, when

Loading required package: rngtools
